In [1]:
import os
import time

import pandas as pd
from ammm_project.grasp import grasp_search
from ammm_project.greedy import greedy_search
from ammm_project.greedy_functions import qs
from ammm_project.local_search import local_search
from ammm_project.parsers.dat import parse
from ammm_project.problem import Problem
from ammm_project.runner import run as runner
from tqdm import tqdm

/Users/vol/UPC/ammm/ammm-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Benchmark mix-instances
knapsack_instances = {}

for filename in os.listdir("../instances"):
    if not filename.endswith("-2.dat"):
        continue

    if not filename.startswith("knapsack"):
        continue

    name, _ = os.path.splitext(filename)

    with open(f"../instances/{filename}", "r") as f:
        dat = parse(f)

    problem = Problem.from_dat(dat)

    knapsack_instances[name] = problem

knapsack_instances

{'knapsack-50-2': Problem(width=1, height=50, max_weight=1569, squares=[Square(id=0, side=1, price=97, weight=17), Square(id=1, side=1, price=38, weight=8), Square(id=2, side=1, price=73, weight=27), Square(id=3, side=1, price=41, weight=54), Square(id=4, side=1, price=60, weight=59), Square(id=5, side=1, price=39, weight=39), Square(id=6, side=1, price=39, weight=81), Square(id=7, side=1, price=82, weight=67), Square(id=8, side=1, price=80, weight=25), Square(id=9, side=1, price=4, weight=78), Square(id=10, side=1, price=94, weight=49), Square(id=11, side=1, price=60, weight=58), Square(id=12, side=1, price=29, weight=100), Square(id=13, side=1, price=67, weight=63), Square(id=14, side=1, price=97, weight=97), Square(id=15, side=1, price=45, weight=60), Square(id=16, side=1, price=90, weight=53), Square(id=17, side=1, price=98, weight=13), Square(id=18, side=1, price=94, weight=91), Square(id=19, side=1, price=64, weight=52), Square(id=20, side=1, price=20, weight=12), Square(id=21, s

In [3]:

bin_packing_instances = {}

for filename in os.listdir("../instances"):
    if not filename.endswith("-2.dat"):
        continue

    if not filename.startswith("bin_packing"):
        continue

    name, _ = os.path.splitext(filename)

    with open(f"../instances/{filename}", "r") as f:
        dat = parse(f)

    problem = Problem.from_dat(dat)

    bin_packing_instances[name] = problem

bin_packing_instances

{'bin_packing-50-2': Problem(width=50, height=100, max_weight=5000, squares=[Square(id=0, side=19, price=361, weight=361), Square(id=1, side=13, price=169, weight=169), Square(id=2, side=6, price=36, weight=36), Square(id=3, side=6, price=36, weight=36), Square(id=4, side=1, price=1, weight=1), Square(id=5, side=1, price=1, weight=1), Square(id=6, side=1, price=1, weight=1), Square(id=7, side=1, price=1, weight=1), Square(id=8, side=1, price=1, weight=1), Square(id=9, side=1, price=1, weight=1), Square(id=10, side=41, price=1681, weight=1681), Square(id=11, side=9, price=81, weight=81), Square(id=12, side=9, price=81, weight=81), Square(id=13, side=9, price=81, weight=81), Square(id=14, side=9, price=81, weight=81), Square(id=15, side=5, price=25, weight=25), Square(id=16, side=4, price=16, weight=16), Square(id=17, side=1, price=1, weight=1), Square(id=18, side=1, price=1, weight=1), Square(id=19, side=1, price=1, weight=1), Square(id=20, side=1, price=1, weight=1), Square(id=21, side

In [4]:
# Benchmark mix-instances
mix_instances = {}

for filename in os.listdir("../instances"):
    if not filename.endswith("-2.dat"):
        continue

    if not filename.startswith("mix"):
        continue

    name, _ = os.path.splitext(filename)

    with open(f"../instances/{filename}", "r") as f:
        dat = parse(f)

    problem = Problem.from_dat(dat)

    mix_instances[name] = problem

mix_instances

{'mix-200-2': Problem(width=200, height=400, max_weight=79200, squares=[Square(id=0, side=126, price=5, weight=29), Square(id=1, side=37, price=85, weight=45), Square(id=2, side=37, price=77, weight=50), Square(id=3, side=37, price=52, weight=98), Square(id=4, side=15, price=68, weight=40), Square(id=5, side=15, price=39, weight=94), Square(id=6, side=7, price=49, weight=54), Square(id=7, side=7, price=26, weight=24), Square(id=8, side=1, price=73, weight=51), Square(id=9, side=1, price=61, weight=75), Square(id=10, side=1, price=31, weight=57), Square(id=11, side=1, price=22, weight=97), Square(id=12, side=1, price=34, weight=26), Square(id=13, side=1, price=44, weight=5), Square(id=14, side=1, price=58, weight=16), Square(id=15, side=6, price=33, weight=8), Square(id=16, side=4, price=25, weight=12), Square(id=17, side=2, price=36, weight=43), Square(id=18, side=2, price=35, weight=86), Square(id=19, side=61, price=18, weight=36), Square(id=20, side=61, price=69, weight=20), Square(i

In [5]:
results = []

In [6]:
for name, problem in tqdm(knapsack_instances.items()):
    # Run Greedy

    time_start = time.time()
    greedy_solution = greedy_search(problem, qs["price_weight"])
    time_end = time.time()

    results.append([name, "greedy", greedy_solution.value, time_end - time_start])

    # Run Local Search
    time_start = time.time()
    greedy_solution = greedy_search(problem, qs["price_weight"])
    local_search_solution = local_search(problem, greedy_solution)
    time_end = time.time()

    results.append(
        (name, "local_search", local_search_solution.value, time_end - time_start)
    )

    # Run GRASP
    def knapsack_grasp(problem):
        return grasp_search(problem=problem, q=qs["price_weight"], alpha=0.1)

    time_start = time.time()

    grasp_solution, iters = runner(
        search_algorithm=knapsack_grasp,
        problem=problem,
        max_time=30 * 60,
        max_time_since_improvement=5 * 60,
    )

    time_end = time.time()

    results.append(
        [
            name,
            "grasp",
            grasp_solution.value if grasp_solution else None,
            time_end - time_start,
        ]
    )

  0%|          | 0/3 [00:00<?, ?it/s]

0.0,0.0,2105


 33%|███▎      | 1/3 [05:00<10:00, 300.01s/it]

1.1920928955078125e-06,1.1920928955078125e-06,9904
0.47935914993286133,0.09587182998657226,9905
0.719635009765625,0.10280500139508929,9906


 67%|██████▋   | 2/3 [10:00<05:00, 300.52s/it]

0.0,0.0,4611


100%|██████████| 3/3 [15:00<00:00, 300.33s/it]


In [7]:
for name, problem in tqdm(bin_packing_instances.items()):
    # Run Greedy

    time_start = time.time()
    greedy_solution = greedy_search(problem, qs["side"])
    time_end = time.time()

    results.append([name, "greedy", greedy_solution.value, time_end - time_start])

    # Run Local Search
    time_start = time.time()
    greedy_solution = greedy_search(problem, qs["side"])
    local_search_solution = local_search(problem, greedy_solution)
    time_end = time.time()

    results.append(
        (name, "local_search", local_search_solution.value, time_end - time_start)
    )

    # Run GRASP
    def bin_packing_grasp(problem):
        return grasp_search(problem=problem, q=qs["side"], alpha=0.2)

    time_start = time.time()

    grasp_solution, iters = runner(
        search_algorithm=bin_packing_grasp,
        problem=problem,
        max_time=30 * 60,
        max_time_since_improvement=5 * 60,
    )

    time_end = time.time()

    results.append(
        [
            name,
            "grasp",
            grasp_solution.value if grasp_solution else None,
            time_end - time_start,
        ]
    )

  0%|          | 0/3 [00:00<?, ?it/s]

9.5367431640625e-07,9.5367431640625e-07,4856
1.4761369228363037,0.14761369228363036,4968


 33%|███▎      | 1/3 [05:01<10:03, 301.74s/it]

7.152557373046875e-07,7.152557373046875e-07,19668
4.235153913497925,1.0587884783744812,19795
36.8007287979126,2.3000455498695374,19874
247.1830756664276,2.3998356860818215,19880


 67%|██████▋   | 2/3 [14:09<07:26, 446.61s/it]

0.0,0.0,79435
227.1441957950592,20.649472345005382,79447
237.86133289337158,19.8217777411143,79468
385.30198192596436,21.405665662553574,79532
419.855331659317,20.99276658296585,79557
438.81824469566345,20.89610689026969,79678


100%|██████████| 3/3 [26:39<00:00, 533.33s/it]


In [8]:
for name, problem in tqdm(mix_instances.items()):
    # Run Greedy

    time_start = time.time()
    greedy_solution = greedy_search(problem, qs["price"])
    time_end = time.time()

    results.append([name, "greedy", greedy_solution.value, time_end - time_start])

    # Run Local Search
    time_start = time.time()
    greedy_solution = greedy_search(problem, qs["price"])
    local_search_solution = local_search(problem, greedy_solution)
    time_end = time.time()

    results.append(
        (name, "local_search", local_search_solution.value, time_end - time_start)
    )

    # Run GRASP
    def mix_grasp(problem):
        return grasp_search(problem=problem, q=qs["price"], alpha=0.3)

    time_start = time.time()

    grasp_solution, iters = runner(
        search_algorithm=mix_grasp,
        problem=problem,
        max_time=30 * 60,
        max_time_since_improvement=5 * 60,
    )

    time_end = time.time()

    results.append(
        [
            name,
            "grasp",
            grasp_solution.value if grasp_solution else None,
            time_end - time_start,
        ]
    )

  0%|          | 0/3 [00:00<?, ?it/s]

0.0,0.0,16907


 33%|███▎      | 1/3 [05:20<10:40, 320.28s/it]

0.0,0.0,7230


 67%|██████▋   | 2/3 [10:21<05:08, 308.95s/it]

0.0,0.0,5982
0.5127017498016357,0.25635087490081787,5983
178.87618494033813,0.5436966107609061,5991
301.8407897949219,0.5448389707489565,5994


100%|██████████| 3/3 [20:23<00:00, 407.88s/it]


In [9]:
df = pd.DataFrame(results, columns=["name", "algorithm", "objective", "time"])
df

,name,algorithm,objective,time
0,knapsack-50-2,greedy,2105,0.001066
1,knapsack-50-2,local_search,2105,0.003247
2,knapsack-50-2,grasp,2105,300.004473
3,knapsack-200-2,greedy,9905,0.002569
4,knapsack-200-2,local_search,9905,0.046536
5,knapsack-200-2,grasp,9906,300.830453
6,knapsack-100-2,greedy,4606,0.001461
7,knapsack-100-2,local_search,4606,0.031766
8,knapsack-100-2,grasp,4611,300.069528
9,bin_packing-50-2,greedy,4952,0.083149


In [11]:
# Save results
#df.to_csv("../results/heuristics-results.csv", index=False)